In [391]:
import pymysql
import spotipy
from tqdm import tqdm

conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='masha999',
    charset='utf8mb4',
    database='playlists'
)

cur = conn.cursor()

## Создание и заполнение таблиц

In [276]:
#токен работает 1 час
token = 'BQBj7nDHGaYcf33OH9K9oJePN4c6innL2loVXC3jUManpip_Af-OAt5XyktHp1NJYPrvrQvdfn2M9PPVUUGi9-xGJymb6cwAmGMsxhpmhop1d6h-I9v8gYiP0L76gLbyBv7hcUV-_owqCwGQecJlEiH52xTCryD-boJsLoQ'
sp = spotipy.Spotify(auth=token)

In [285]:
cur.execute('''CREATE TABLE IF NOT EXISTS track_playlist (id_t_p int AUTO_INCREMENT NOT NULL, track_id varchar(255), playlist_id varchar(255), track_number int, PRIMARY KEY(id_t_p))''')
cur.execute('''CREATE TABLE IF NOT EXISTS track_artist (id_t_ar int AUTO_INCREMENT NOT NULL, track_id varchar(255), artist_id varchar(255), PRIMARY KEY(id_t_ar))''')
cur.execute('''CREATE TABLE IF NOT EXISTS album_artist (id_al_ar int AUTO_INCREMENT NOT NULL, album_id varchar(255), artist_id varchar(255), PRIMARY KEY(id_al_ar))''')
cur.execute('''CREATE TABLE IF NOT EXISTS playlists (playlist_id varchar(255) NOT NULL, name varchar(256), followers int, total_tracks int, PRIMARY KEY(playlist_id), UNIQUE(playlist_id))''')
cur.execute('''CREATE TABLE IF NOT EXISTS tracks (track_id varchar(255) NOT NULL, name varchar(256), duration int, popularity int, album_id varchar(256), explicit int, PRIMARY KEY(track_id), UNIQUE(track_id))''')
cur.execute('''CREATE TABLE IF NOT EXISTS albums (album_id varchar(255) NOT NULL, name varchar(256), total_tracks varchar(256), release_date date, PRIMARY KEY(album_id), UNIQUE(album_id))''')
cur.execute('''CREATE TABLE IF NOT EXISTS artists (artist_id varchar(255) NOT NULL, name varchar(256), PRIMARY KEY(artist_id), UNIQUE(artist_id))''')
conn.commit()

In [286]:
pls = sp.current_user_playlists()
inserted = []
for pl in tqdm(pls['items']):
    pl = sp.playlist(pl['id'])
    pl_id = pl['id']
    pl_name = pl['name']
    pl_tr = pl['tracks']['items']
    pl_num_tr = pl['tracks']['total']
    pl_fl = pl['followers']['total']
    cur.execute(f'INSERT IGNORE INTO playlists (playlist_id, name, followers, total_tracks) VALUES ("{pl_id}", "{pl_name}", {pl_fl}, {pl_num_tr})')
    for i, tr in enumerate(pl_tr):
        tr_id = tr['track']['id']
        tr_dur = tr['track']['duration_ms']
        tr_pop = tr['track']['popularity']
        tr_exp = int(tr['track']['explicit'])
        tr_name = tr['track']['name']
        tr_art = tr['track']['artists']
        tr_n = i + 1
        alb = tr['track']['album']
        alb_id = alb['id']
        alb_tp = alb['type']
        alb_name = alb['name']
        alb_num_tr = alb['total_tracks']
        alb_dt = alb['release_date']
        alb_art = alb['artists']
        cur.execute(f'INSERT IGNORE INTO track_playlist (track_id, playlist_id, track_number) VALUES ("{tr_id}", "{pl_id}", {tr_n})')
        if tr_id not in inserted:
            cur.execute(f'INSERT IGNORE INTO tracks (track_id, name, popularity, duration, explicit, album_id) VALUES ("{tr_id}", "{tr_name}", {tr_pop}, {tr_dur}, {tr_exp}, "{alb_id}")')
            cur.execute(f'INSERT IGNORE INTO albums (album_id, name, total_tracks, release_date) VALUES ("{alb_id}", "{alb_name}", {alb_num_tr}, "{alb_dt}")')
            for art in tr_art:
                art_id = art['id']
                art_name = art['name']
                cur.execute(f'INSERT IGNORE INTO artists (artist_id, name) VALUES ("{art_id}", "{art_name}")')
                cur.execute(f'INSERT IGNORE INTO track_artist (track_id, artist_id) VALUES ("{tr_id}", "{art_id}")')
            for art in alb_art:
                art_id = art['id']
                art_name = art['name']
                cur.execute(f'INSERT IGNORE INTO artists (artist_id, name) VALUES ("{art_id}", "{art_name}")')
            cur.execute(f'INSERT IGNORE INTO album_artist (album_id, artist_id) VALUES ("{alb_id}", "{art_id}")')
            inserted.append(tr_id)
            
conn.commit()

100%|██████████| 15/15 [00:04<00:00,  3.60it/s]


## Запросы

1. Топ-10 альбомов по средней популярности треков

In [287]:
cur.execute('''SELECT albums.name, AVG(popularity) FROM tracks 
            INNER JOIN albums ON tracks.album_id = albums.album_id 
            GROUP BY tracks.album_id ORDER BY AVG(popularity) DESC LIMIT 10''')
cur.fetchall()

(('Therefore I Am', Decimal('82.0000')),
 ('The Truth About Love', Decimal('80.0000')),
 ('ITAEWON CLASS (Original Television Soundtrack) Pt. 12', Decimal('80.0000')),
 ('Film out', Decimal('80.0000')),
 ('WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?', Decimal('79.0000')),
 ('Raise Your Glass', Decimal('79.0000')),
 ('MAP OF THE SOUL : PERSONA', Decimal('78.6667')),
 ('eight', Decimal('77.0000')),
 ('Savage Love (Laxed - Siren Beat) [BTS Remix]', Decimal('77.0000')),
 ('THE ALBUM', Decimal('77.0000')))

2. Трек, название плейлиста, позиция трека в плейлисте

In [296]:
cur.execute('''SELECT tracks.name, playlists.name, track_playlist.track_number 
            FROM playlists, tracks, track_playlist 
            WHERE tracks.track_id = track_playlist.track_id AND playlists.playlist_id = track_playlist.playlist_id
            LIMIT 10''')
cur.fetchall()

(('ариведерчи', 'Ru-playlist', 1),
 ('Ведьма III - IV', 'Ru-playlist', 2),
 ('Новые люди', 'Ru-playlist', 3),
 ('Формалин', 'Ru-playlist', 4),
 ('Впусти музыку', 'Ru-playlist', 5),
 ('А мы не ангелы, парень', 'Ru-playlist', 6),
 ('Всадник из льда', 'Ru-playlist', 7),
 ('Я выбираю ночь', 'Ru-playlist', 8),
 ('Ведьма 3', 'Ru-playlist', 9),
 ('Одиночка', 'Ru-playlist', 10))

3. Топ-5 исполнителей по количеству треков в плейлистах

In [297]:
cur.execute('''SELECT artists.name, art_tr_count.artist_count FROM artists INNER JOIN
                (SELECT track_artist.artist_id, COUNT(*) AS artist_count FROM track_artist 
                GROUP BY track_artist.artist_id) art_tr_count 
                ON artists.artist_id = art_tr_count.artist_id ORDER BY artist_count DESC LIMIT 5''')
cur.fetchall()

(('Pentatonix', 130), ('BTS', 83), ('RM', 12), ('Agust D', 8), ('Halsey', 7))

4. Треки, которые исполняют больше 2 артистов

In [396]:
cur.execute('''SELECT tracks.name, COUNT(artists.name) FROM track_artist 
            INNER JOIN tracks ON tracks.track_id = track_artist.track_id 
            INNER JOIN artists ON artists.artist_id = track_artist.artist_id 
            GROUP BY track_artist.track_id HAVING COUNT(artists.name) > 2 ORDER BY COUNT(artists.name) ASC''')
cur.fetchall()

(('Хоккей', 3),
 ("SUGA's Interlude", 3),
 ("SUGA's Interlude", 3),
 ('От винта!', 3),
 ("It Can't Be True", 3),
 ('Savage Love (Laxed – Siren Beat) [BTS Remix]', 3),
 ('Вести-бу-би-лю-би-бу', 3),
 ('Crying Over You ◐ (feat. RM & BEKA)', 3),
 ('Бедненький больной', 4),
 ('Город Омск', 4))

5. Альбомы, выпущенные в один день (попарно)

In [397]:
cur.execute('''SELECT A.name, B.name, A.release_date FROM albums A, albums B 
            WHERE A.album_id != B.album_id AND A.release_date = B.release_date
            ORDER BY A.release_date DESC''')
cur.fetchall()

(('minisode1 : Blue Hour', 'Eyes wide open', datetime.date(2020, 10, 26)),
 ('Eyes Wide Open', 'minisode1 : Blue Hour', datetime.date(2020, 10, 26)),
 ('Eyes Wide Open', 'Eyes wide open', datetime.date(2020, 10, 26)),
 ('Eyes wide open', 'minisode1 : Blue Hour', datetime.date(2020, 10, 26)),
 ('minisode1 : Blue Hour', 'Eyes Wide Open', datetime.date(2020, 10, 26)),
 ('Eyes wide open', 'Eyes Wide Open', datetime.date(2020, 10, 26)),
 ('Savage Love (Laxed - Siren Beat) [BTS Remix]',
  'Mad World',
  datetime.date(2020, 10, 2)),
 ('THE ALBUM', 'Mad World', datetime.date(2020, 10, 2)),
 ('Mad World', 'THE ALBUM', datetime.date(2020, 10, 2)),
 ('Mad World',
  'Savage Love (Laxed - Siren Beat) [BTS Remix]',
  datetime.date(2020, 10, 2)),
 ('Savage Love (Laxed - Siren Beat) [BTS Remix]',
  'THE ALBUM',
  datetime.date(2020, 10, 2)),
 ('THE ALBUM',
  'Savage Love (Laxed - Siren Beat) [BTS Remix]',
  datetime.date(2020, 10, 2)),
 ('Dynamite (NightTime Version)', 'Colour Vision', datetime.date(2

6. Треки, популярность которых выше средней популярности тех треков, у которых популярность больше 0

In [377]:
cur.execute('''SELECT tracks.name, popularity FROM tracks 
            WHERE popularity > (SELECT AVG(popularity) FROM tracks WHERE popularity > 0)
            ORDER BY popularity''')
cur.fetchall()

(('Bohemian Rhapsody', 57),
 ('P.O.P (Piece Of Peace) Pt. 1', 57),
 ('Cheerleader (OMI Cover)', 57),
 ('Reflection', 57),
 ('Shallow', 57),
 ("That's Christmas to Me", 58),
 ('Dynamite', 58),
 ('Deck The Halls', 58),
 ('Skit', 58),
 ('Jolene (feat. Dolly Parton)', 58),
 ('Can You Feel the Love Tonight', 58),
 ('Stay Here', 59),
 ('Mary, Did You Know?', 59),
 ('In Silence', 59),
 ('badbye', 59),
 ('GO HARD', 59),
 ('Little Drummer Boy', 59),
 ('Interlude: Wings', 59),
 ('Never Grow Up', 59),
 ('Crying Over You ◐ (feat. RM & BEKA)', 59),
 ('Change', 60),
 ('First Love', 60),
 ('Interlude : Set me free', 60),
 ('Carol of the Bells', 60),
 ('Honsool', 60),
 ("SUGA's Interlude", 61),
 ('Attack on Bangtan', 61),
 ("Please Don't Leave Me", 61),
 ('Sing', 61),
 ('Формалин', 61),
 ('Look Here', 61),
 ('Coldest Winter - Bobby Alt Drum Mix', 61),
 ('Love me or Leave me', 61),
 ('uhgood', 61),
 ('God Rest Ye Merry Gentlemen', 62),
 ("Winter Wonderland / Don't Worry Be Happy (feat. Tori Kelly)", 62

7. Кол-во explicit треков в альбомах

In [392]:
#cur.execute('ALTER TABLE albums DROP COLUMN num_exp')

0

In [393]:
cur.execute('ALTER TABLE albums ADD num_exp int')
cur.execute('''UPDATE albums, (SELECT album_id, SUM(explicit) AS exp_al FROM tracks GROUP BY album_id) AS al_ex
            SET num_exp = al_ex.exp_al WHERE albums.album_id = al_ex.album_id''')
cur.execute('SELECT name, num_exp FROM albums WHERE num_exp > 0')
cur.fetchall()


(('D-2', 2),
 ('Funhouse (Expanded Edition)', 1),
 ('Greatest Hits...So Far!!!', 1),
 ('Raise Your Glass', 1),
 ('The Most Beautiful Moment in Life Pt.2', 1),
 ('Nightmare', 1),
 ('Savage Love (Laxed - Siren Beat) [BTS Remix]', 1))

In [389]:
conn.commit()

In [390]:
conn.close()